# Prediction based on Trailer

## Import

In [0]:
import json
import os
import pickle
import urllib
import os
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

try:
    from google.colab import drive
    drive_dir = '/content/drive'
    drive.mount(drive_dir)
    root_dir = '/content/drive/My Drive/AML/'
    git_dir = root_dir+'Git/'
    COLAB_IN = True
except:
    COLAB_IN = False

if COLAB_IN:
    os.chdir("drive/My Drive/AML/Git_lastClone/neural-network/trailer_model/visual_analyzer")
    !pip install youtube_dl
    !pip install imdbpy
    !pip install bcolz
    
import keras
from keras.models import Sequential
import numpy as np
import tensorflow as tf

import json
import functools
import itertools
import numpy
import operator
from keras.utils import to_categorical
import urllib.parse
import urllib.request
import requests

import youtube_dl

import imdb
from tqdm import tnrange, tqdm_notebook, tqdm
import sys
from bs4 import BeautifulSoup

import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images

import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
import time


import os 
import argparse
import time
from __future__ import print_function
import argparse
import cv2
import os 
import PIL
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.regularizers import l2
import numpy as np
import pickle as cPickle 
import numpy
import cv2
import scipy
import csv
import imutils
from skimage import io
import dlib
import json
import time
import pandas as pd 
import glob
from IPython.display import Image


## Utility functions

In [0]:
def urlopen(url, mobile = False):
    try:
        if mobile:
            urlheader =  {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 5_0 like Mac OS X) AppleWebKit/534.46' ,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
        else:
            urlheader = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                          'AppleWebKit/537.11 (KHTML, like Gecko) '
                          'Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'
                        }
        #header2 = 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17'
        return urllib.request.urlopen(urllib.request.Request(url=url, data=None, headers=urlheader)).read().decode('utf-8')
    except HTTPError as e:
        if (_WARNINGS):
            time.sleep(5);
            warnings.warn(str(e))
            return urlopen(url)
        else:
            raise e

## Dataset

### Loader

In [0]:
class Entry:
  
  def __init__(self, **kwargs):
    self.movie_id = kwargs['movie_id']
    self.name = kwargs['name']
    self.revenue_opening = kwargs['revenue_opening']
    self.revenue_total = kwargs['revenue_total']
    
  def __repr__(self):
    return f'Name: {self.name}; movie ID: {self.movie_id}'
    
  def set_metadata(self, metadata_dict):
    self.year = metadata_dict['year']
    self.genres = metadata_dict['genres']
    self.actors = metadata_dict['actors']
    self.directors = metadata_dict['directors']
    self.creators = metadata_dict['creators']
    self.duration = metadata_dict['duration']
    
  def set_trailers(self, trailer_dict):
    self.imdb_trailer = trailer_dict['imdb_trailer']
    self.youtube_trailer = trailer_dict['youtube_trailer']
    

class Dataset:
  
  def __init__(self, dataset_dir):
    self.basic_data = pd.read_csv(os.path.join(dataset_dir, 'Dataset.csv'))
    self.dataset_dir = dataset_dir
    
    self.entries = list()
    self.movie_indeces = dict()
    self._create_entries()
    
    self.Youtube_urlroot = "https://www.youtube.com"
    self.Imdb_urlroot = "https://www.imdb.com"
    
  def get_entry(self, movie_id):
    return self.entries[self.movie_indeces[movie_id]]
  
  def get_revenue(self, movie_id):
    tmp_df = self.basic_data[self.basic_data['movie_id'] == movie_id]
    return tmp_df['revenue_opening'].values[0]
  
  def load_metadata(self):
    with open(os.path.join(self.dataset_dir, 'metadata.data'), 'rb') as fr:
      metadata_dict = pickle.load(fr)
      
    for k, v in metadata_dict['global'].items():
      self.__dict__[k] = self._decompress_global_categorical(v)
      
    for movie_id, movie_metadata in metadata_dict['entries'].items():
      entry = self.entries[self.movie_indeces[movie_id]]
      entry.set_metadata(movie_metadata)
      
      # decompress categorical
      entry.actors = self._decompress_categorical(entry.actors, len(self.all_actors))
      entry.creators = self._decompress_categorical(entry.creators, len(self.all_creators))
      entry.directors = self._decompress_categorical(entry.directors, len(self.all_directors))
      entry.genres = self._decompress_categorical(entry.genres, len(self.all_genres))
    
  def load_trailers(self):
    with open(os.path.join(self.dataset_dir, 'trailers.data'), 'rb') as fr:
      trailers_dict = pickle.load(fr)
      
    for movie_id, trailers in trailers_dict.items():
      entry = self.entries[self.movie_indeces[movie_id]]
      entry.set_trailers(trailers)
  
  def GetTrailer(self, IMDbVideoUrl, YoutubeVideoUrl, trailers_dir, filename): # download trailer
        file_output = os.path.join(trailers_dir, filename+".mp4")
        !youtube-dl -f best --force-ipv4 --merge-output-format mp4 '$YoutubeVideoUrl' -o '$file_output' > /dev/null
            
  def _create_entries(self):
    for idx, [movie_id, name, rev_open, rev_total] in self.basic_data.iterrows():
      entry = Entry(movie_id=movie_id, name=name,
                   revenue_opening = rev_open,
                   revenue_total = rev_total)
      self.entries.append(entry)
      self.movie_indeces[movie_id] = idx
      
  def _compress_global_categorical(self, input_dict):
    output_dict = dict()
    for k, v in input_dict.items():
      output_dict[k] = np.argmax(v)

    return output_dict
  
  def _decompress_global_categorical(self, input_dict):
    output_dict = dict()
    for k, v in input_dict.items():
      indicator = np.zeros(len(input_dict))
      indicator[v] = 1
      output_dict[k] = indicator
      
    return output_dict
  
  def _compress_categorical(self, arr):
    dct = dict()
    for idx, val in enumerate(arr):
      if val > 0:
        dct[idx] = val

    return dct
  
  def _decompress_categorical(self, arr, total_len):
    indicator = np.zeros(total_len)
    for idx, val in arr.items():
      indicator[idx] = val
    
    return indicator

In [0]:
movies = Dataset(dataset_dir = '../../dataset/')

movies.load_trailers()
movies.load_metadata()

##Feature Extractor

In [0]:

class VisualFeatureExtractor:
    def __init__(self, model_dir):
        self.emotionPredictionModel(model_dir)
        self.detector = dlib.get_frontal_face_detector()
    
    
    # ****** image processing ****** """
    def Flip(self, data):
        dataFlipped = data[..., ::-1].reshape(2304).tolist()
        return dataFlipped

    def Roated15Left(self, data):
        num_rows, num_cols = data.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), 20, 1)
        img_rotation = cv2.warpAffine(data, rotation_matrix, (num_cols, num_rows))
        return img_rotation.reshape(2304).tolist()
    
    def Roated15Right(self, data):
        num_rows, num_cols = data.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), -20, 1)
        img_rotation = cv2.warpAffine(data, rotation_matrix, (num_cols, num_rows))
        return img_rotation.reshape(2304).tolist()

    def shiftedUp20(self, data):
        translated = imutils.translate(data, 0, -5)
        translated2 = translated.reshape(2304).tolist()
        return translated2
    
    def shiftedDown20(self, data):
        translated = imutils.translate(data, 0, 5)
        translated2 = translated.reshape(2304).tolist()
        return translated2

    def shiftedLeft20(self, data):
        translated = imutils.translate(data, -5, 0)
        translated2 = translated.reshape(2304).tolist()
        return translated2
    
    def shiftedRight20(self, data):
        translated = imutils.translate(data, 5, 0)
        translated2 = translated.reshape(2304).tolist()
        return translated2

    def flatten_matrix(self, matrix):
        vector = matrix.flatten(1)
        vector = vector.reshape(1, len(vector))
        return vector
    
    def zca_whitening(self, inputs):
        sigma = np.dot(inputs, inputs.T)/inputs.shape[1] #Correlation matrix
        U,S,V = np.linalg.svd(sigma) #Singular Value Decomposition
        epsilon = 0.1                #Whitening constant, it prevents division by zero
        ZCAMatrix = np.dot(np.dot(U, np.diag(1.0/np.sqrt(np.diag(S) + epsilon))), U.T)                     #ZCA Whitening matrix
        return np.dot(ZCAMatrix, inputs)   #Data whitening
    
    def global_contrast_normalize(self, X, scale=1., subtract_mean=True, use_std=True, sqrt_bias=10, min_divisor=1e-8):
        assert X.ndim == 2, "X.ndim must be 2"
        scale = float(scale)
        assert scale >= min_divisor
        mean = X.mean(axis=1)
        if subtract_mean:
            X = X - mean[:, numpy.newaxis]  # Makes a copy.
        else:
            X = X.copy()

        if use_std:
            ddof = 1
            if X.shape[1] == 1:
                ddof = 0

            normalizers = numpy.sqrt(sqrt_bias + X.var(axis=1, ddof=ddof)) / scale
        else:
            normalizers = numpy.sqrt(sqrt_bias + (X ** 2).sum(axis=1)) / scale

        # Don't normalize by anything too small.
        normalizers[normalizers < min_divisor] = 1.

        X /= normalizers[:, numpy.newaxis]  # Does not make a copy.
        return X
    
    def ZeroCenter(self, data):
        data = data - numpy.mean(data,axis=0)
        return data

    def normalize(self, arr):
        for i in range(3):
            minval = arr[...,i].min()
            maxval = arr[...,i].max()
            if minval != maxval:
                arr[...,i] -= minval
                arr[...,i] *= (255.0/(maxval-minval))
        return arr

    def ConvertToArrayandReshape(self, List):
        numpyarray = numpy.asarray(List)
        numpyarray = numpyarray.reshape(1,48,48)
        numpyarray = numpyarray.reshape(1, 48, 48,1)
        numpyarray = numpyarray.astype('float32')
        return numpyarray
    
    def imagePreprocessing(self, crop2):
        data2 = self.ZeroCenter(crop2)
        data3 = self.zca_whitening(self.flatten_matrix(data2)).reshape(48,48)
        data4 = self.global_contrast_normalize(data3)
        data5 = numpy.rot90(data4,3)
        return data5
    
    #calculate brightness
    def getBrightness(self, frame):   
        import cv2
        #convert image to HSV format
        HSV=cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)   
        #calculate the brightness (mean of the V-Channel)
        brightness = np.mean(HSV[:,:,2].flatten())
        return brightness


    #calculate sharpness
    def getSharpness(self, frame):
        #install cpbd library and its dependies 
        !pip install cpbd
        !pip install scipy==4.3.0
        !pip install Pillow

        import cv2
        import cpbd
        from scipy import ndimage
        import matplotlib.pyplot as plt
        #estimate sharpness
        sharpness=cpbd.compute(frame)
        return sharpness
    
     # ****** Emotion prediction (NN) ****** """
    def emotionPredictionModel(self, model_dir):
        img_rows, img_cols = 48, 48
        # the CIFAR10 images are RGB
        img_channels = 1

        model = Sequential()
        model.add(Convolution2D(64, (5, 5), border_mode='valid',
                                input_shape=(img_rows, img_cols,1)))
        model.add(keras.layers.advanced_activations.PReLU(alpha_initializer='zero', weights=None))
        model.add(keras.layers.convolutional.ZeroPadding2D(padding=(2, 2), data_format="channels_first"))
        model.add(MaxPooling2D(pool_size=(5, 5),strides=(2, 2)))

        model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), data_format="channels_first")) 
        model.add(Convolution2D(64, (3, 3)))
        model.add(keras.layers.advanced_activations.PReLU(alpha_initializer='zero', weights=None))
        model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), data_format="channels_first")) 
        model.add(Convolution2D(64, (3, 3)))
        model.add(keras.layers.advanced_activations.PReLU(alpha_initializer='zero', weights=None))
        model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))

        model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), data_format="channels_first"))
        model.add(Convolution2D(128, (3, 3)))
        model.add(keras.layers.advanced_activations.PReLU(alpha_initializer='zero', weights=None))
        model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), data_format="channels_first"))
        model.add(Convolution2D(128, (3, 3)))
        model.add(keras.layers.advanced_activations.PReLU(alpha_initializer='zero', weights=None))

        model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), data_format="channels_first"))
        model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))


        model.add(Flatten())
        model.add(Dense(1024))
        model.add(keras.layers.advanced_activations.PReLU(alpha_initializer='zero', weights=None))
        model.add(Dropout(0.2))
        model.add(Dense(1024))
        model.add(keras.layers.advanced_activations.PReLU(alpha_initializer='zero', weights=None))
        model.add(Dropout(0.2))


        model.add(Dense(7))


        model.add(Activation('softmax'))


        ada = Adadelta(lr=0.1, rho=0.95, epsilon=1e-08)
        model.compile(loss='categorical_crossentropy',
                      optimizer=ada,
                      metrics=['accuracy'])


        filepath=os.path.join(model_dir, "Model.120-0.6343.hdf5")
        print(filepath)
        model.load_weights(filepath)
        self.model = model
        
    def emotionPrediction(self, data5):
        Train_x_Init = self.ConvertToArrayandReshape(data5)
        Train_x_Flip  = self.ConvertToArrayandReshape(self.Flip(data5))
        Train_x_Rotleft = self.ConvertToArrayandReshape(self.Roated15Left(data5))
        Train_x_Rotright = self.ConvertToArrayandReshape(self.Roated15Right(data5))
        Train_x_ShiftedUp = self.ConvertToArrayandReshape(self.shiftedUp20(data5))
        Train_x_ShiftedDown = self.ConvertToArrayandReshape(self.shiftedDown20(data5))
        Train_x_ShiftedLeft = self.ConvertToArrayandReshape(self.shiftedLeft20(data5))
        Train_x_ShiftedRight = self.ConvertToArrayandReshape(self.shiftedRight20(data5))
        a= np.array([  self.model.predict_proba(Train_x_Init, verbose=0)[0]])
        return a.mean(axis=0)
    
    # ****** Face detection ****** """
    def faceDetection(self, img):#extract faces
        grayimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        dets, scores, idx= self.detector.run(grayimg,1)
        imgWithRect = img#cv2.imread(imagepath)
        ListOfFaces = []
        FaceWriteList = []
        temp=0
        for rectangle in dets:
            if(scores[temp]>0.2):
                #print("face detected")
                left = rectangle.left()
                top = rectangle.top()
                right = rectangle.right()
                bottom = rectangle.bottom()
                if top<0:
                    offset = 0-top
                    top = 0
                    bottom = bottom+offset
                if left<0:
                    offset = 0 -left
                    left =0
                    right = right+offset
                cv2.rectangle(imgWithRect,(left,top),(right,bottom),(255,255,255),2)
                crop = grayimg[top:bottom, left:right]
                colored_crop=img[top:bottom, left:right]
                crop2 = cv2.resize(crop, (48, 48)) 
                data = self.imagePreprocessing(crop2)
                ListOfFaces.append(data)
                FaceWriteList.append(colored_crop)
            temp+=1

        return ListOfFaces, FaceWriteList
    
    
    # ****** extraction ****** """
    def extract(self, file, resolution):
        
        import csv
        #fps
        fps=1
        #read the video from the file
        cap = cv2.VideoCapture(file)
        cap.set(cv2.CAP_PROP_FPS, 15)
        n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # to add to csv
        video_fps = cap.get(cv2.CAP_PROP_FPS)
        width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
        height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float
        try:
            duration = n_frames/video_fps
        except:
            duration=0.0
        print('duration=',duration)
        #initialization
        colors_R = []
        colors_G = []
        colors_B = []
        brightness=[]
        sharpness=[]
        Anger=[0.0]
        Disgust=[0.0]
        Fear=[0.0]
        Happy=[0.0]
        Sad=[0.0]
        Surprise=[0.0]
        Neutral=[0.0]
        
        ret = True
        frame_index = -1
        count=0
        
        pbar2 = tqdm_notebook(total=n_frames)
        if duration<=600:
            while ret:
                count=count+1
                #next frame
                frame_index = frame_index + 1   
                #read the frame
                ret, frame = cap.read()


                if video_fps > fps and frame_index % int(video_fps / fps) != 0:
                    continue

                if ret:         
                    #extract colors
                    colors_R.append(np.mean(frame[:,:,0].flatten()))
                    colors_G.append(np.mean(frame[:,:,1].flatten()))
                    colors_B.append(np.mean(frame[:,:,2].flatten()))
                    #extract brightness
                    brightness.append(self.getBrightness(frame))
                    #extract sharpness
                    #sharpness.append(self.getSharpness(frame))
                    #integrate the code of syrine to extract emotion
                    if count%10==0:

                        ListOfFaces,FaceWriteList = self.faceDetection(frame)
                        temp_list=[]
                        for face, coors in zip(ListOfFaces, FaceWriteList):
                            result = self.emotionPrediction(face)
                            temp_list.append(result)
                        for y in temp_list:
                            Anger.append(y[0])
                            Disgust.append(y[1])
                            Fear.append(y[2])
                            Happy.append(y[3])
                            Sad.append(y[4])
                            Surprise.append(y[5])
                            Neutral.append(y[6])
                pbar2.update(frame_index - pbar2.n)
        
            
            
            mean_Anger=np.mean(np.array(Anger))
            mean_Disgust=np.mean(np.array(Disgust))
            mean_Fear=np.mean(np.array(Fear))
            mean_Happy=np.mean(np.array(Happy))
            mean_Sad=np.mean(np.array(Sad))
            mean_Surprise=np.mean(np.array(Surprise))
            mean_Neutral=np.mean(np.array(Neutral))

            var_Anger=np.var(np.array(Anger))
            var_Disgust=np.var(np.array(Disgust))
            var_Fear=np.var(np.array(Fear))
            var_Happy=np.var(np.array(Happy))
            var_Sad=np.var(np.array(Sad))
            var_Surprise=np.var(np.array(Surprise))
            var_Neutral=np.var(np.array(Neutral))

            list_to_append=[mean_Anger,var_Anger,mean_Disgust,var_Disgust,mean_Fear,var_Fear,mean_Happy,var_Happy,mean_Sad,var_Sad,mean_Surprise,var_Surprise,mean_Neutral,var_Neutral,np.mean(np.array(colors_R)),np.var(np.array(colors_R)),np.mean(np.array(colors_G)),np.var(np.array(colors_G)),np.mean(np.array(colors_B)),np.var(np.array(colors_B)),np.mean(np.array(brightness)),np.mean(np.var(brightness)),video_fps,width,height,duration]
            #df=df.append(pd.Series(list_to_append,index=["IMDB_ID","mean_Anger","var_Anger","mean_Disgust","var_Disgust","mean_Fear","var_Fear","mean_Happy","var_Happy","mean_Sad","var_Sad","mean_Surprise","var_Surprise","mean_Neutral","var_Neutral","R_mean","R_var","G_mean","G_var","B_mean","B_var","Brightness_mean","Brightness_var","Sharpness_mean","Sharpness_var"]),ignore_index=True)
        
        else:
             list_to_append=[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
        return list_to_append
                                                                                                                                                                                                                                                      

    # ****** iteration over all movies ****** """
    def run(self, movies, trailers_dir, resolution, output_path, output_file):

        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        features_name = ["IMDB_ID","mean_Anger","var_Anger","mean_Disgust","var_Disgust","mean_Fear","var_Fear","mean_Happy","var_Happy","mean_Sad","var_Sad","mean_Surprise","var_Surprise","mean_Neutral","var_Neutral","R_mean","R_var","G_mean","G_var","B_mean","B_var","Brightness_mean","Brightness_var","video_fps","width","height","duration"]
        #df =pd.DataFrame(columns=features_name)
        '''with open('Akram.csv', 'w+') as fp:
            wr = csv.writer(fp)
            wr.writerow(features_name)'''
            
        #iterate over videos
        movies_to_iterate = movies.entries[6000+20:7000]
        pbar1 = tqdm_notebook(total=len(movies_to_iterate))
        
        for id, entry in enumerate(movies_to_iterate):# start:end
            # Store sample
            videoFile = os.path.join(trailers_dir, entry.movie_id +'.mp4')
            print('Retrieving movie ', entry.movie_id, 'id:', id)
            #print('Retrieving movie ', entry.movie_id)
            broken = False
            if not os.path.isfile(videoFile):
                print('Donwload trailer')
                movies.GetTrailer(entry.imdb_trailer, entry.youtube_trailer, trailers_dir, entry.movie_id)
                #time.sleep(5)
                timer = time.time()
                while not os.path.isfile(videoFile):
                    #print('Trailer is not yet ready')
                    time.sleep(1)
                    if(time.time() - timer > 20):
                        broken = True
                        break
            if broken:
                continue
            """if not os.path.isfile(videoFile):
                print('Donwload trailer', id)
                print(entry.youtube_trailer)
                movies.GetTrailer(entry.imdb_trailer, entry.youtube_trailer, trailers_dir, entry.movie_id)
                
            while not os.path.isfile(videoFile):
                #print('Trailer is not yet ready')
                time.sleep(1)"""
            print('Start extracting visual features')
            # video to frames and append to colors list
            list_to_append=[entry.movie_id, ] + self.extract(os.path.join(trailers_dir, entry.movie_id+'.mp4'), resolution)
            #df=df.append(pd.Series(list_to_append,index=features_name),ignore_index=True)
            with open(os.path.join(output_path,output_file), 'a',newline='') as f:
                wr = csv.writer(f)
                wr.writerow(list_to_append)
            print('Extracting features done!')
            #delete video
            file_to_remove = os.path.join(trailers_dir, entry.movie_id+'.mp4')
            !rm -r $file_to_remove > /dev/null
            !rm -rf ~/.local/share/Trash/*
            #!rm -r output_path+entry.movie_id+'.mp4
            #!rm -r to_remove
            pbar1.update(1)

        #df.to_csv(os.path.join(output_path, output_file), sep=',')
        print("Sucess, Completed")

In [0]:
visual_feature_extractor = VisualFeatureExtractor(model_dir = '../trained_model')
visual_feature_extractor.run(movies = movies, trailers_dir = "./", resolution = (640, 480), output_path = "./", output_file ="visual_features.csv")